# Convolutional Neural Networks

In [ ]:
!wget "http://dl.dropboxusercontent.com/s/w9aqbqxmj4i2my8/dataset.zip"
!unzip dataset.zip
!ls

## Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator  

In [ ]:
tf.__version__

'2.8.0'

# Part 1 - Data Preprocessing

## Preprocessing the training set

In [ ]:
# we need to do some trasnformations as preprocessing on training set for reducing overfitting

# For this some geometric trasnformations are applied such as zoom, rotations, shift pixels horizontal flips and some zoom in and zoom out
# The tecnical term for transforming the images of training set is called Image Augmentation
# This will prevent CNN from overlearning or overfitting

train_datagen = ImageDataGenerator(
                  rescale=1./255,     # performs feature scaling and acts as standardization to get values between 0 and 1 as there are 255 values for color
                  shear_range=0.2,
                  zoom_range=0.2,
                  horizontal_flip=True
)

training_set = train_datagen.flow_from_directory(
    'dataset/training_set', # path to training set
    target_size=(64,64), # final size of image set which is going to be fed to CNN. larger values takes more time
    batch_size=32, # how amny images in each batch. 32 is a common value. making batches reduces the load on machine
    class_mode= 'binary' # specifies either binary or categorical. as now we have to choose betrwenn cat and dog hence binary 
)



Found 8000 images belonging to 2 classes.


## Preprocessing the test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./200) # we are not going to apply the transformations we did on training set as they are new images 
                                # which are going to passed when deploying and need to keep them intact. 
                                # But rescaling must be done (i.e feature scaling as both training sand test set should be in same scale)

test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),   # same size as used for training set
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


# Part 2 : Building the CNN

## Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()    # to initilize cnn variable as sequence of layers. other is computational graph

## Step 1 : Convolution

In [ ]:
# the convolutional layer going to be added below using add method is an object of class Conv2D
# this class acts as the dense class which allows us to build fully connected layer

cnn.add(tf.keras.layers.Conv2D(       #  to add concolution layer
    filters = 32,                      # no of feture detectors (also called filters or kernels) which need to be applied,
    kernel_size = 3,                   # size of feature detector/kernel i.e size of row/ col as it is a sq matrix
    activation = 'relu',
    input_shape = [64,64,3]         # as we have resized the image to 64 x 64, it should be the size. and since it is color, we need to specify channel as 3. if grayscale it should be 1    
    ))

## Step 2 : Pooling

In [ ]:
# Applying max pooling (refer doc)
# as we are adding pooling layer to our CNN, we need to call add method which is used to add layers in NN

# here also we are passing an obkect of class Maxpool2D belong to layers of tf.keras as shown above for max pooling
cnn.add(tf.keras.layers.MaxPool2D(
      pool_size = 2,          # size of frame which should be considered which is asq matrix (refer doc for ex, same is considered here)
      strides = 2,        # specifies no of  pixels for frame to shift. it acts as an offset for jumping of frame from one block to next block. 2 means after 1 ele, it jusmps to 3rd ele
      # check other args also
))

## Adding a second Convolution Layer

In [ ]:
# we need to do the above steps again for second concplution layer

cnn.add(tf.keras.layers.Conv2D(       #  to add concolution layer
    filters = 32,                      # no of feture detectors (also called filters or kernels) which need to be applied,
    kernel_size = 3,                   # size of feature detector/kernel i.e size of row/ col as it is a sq matrix
    activation = 'relu',
    # we dont need to specify input_size, its needed only for first time to connect automatically connect input layer to first layer0 
    ))

cnn.add(tf.keras.layers.MaxPool2D(
      pool_size = 2,          # size of frame which should be considered which is asq matrix (refer doc for ex, same is considered here)
      strides = 2,        # specifies no of  pixels for frame to shift. it acts as an offset for jumping of frame from one block to next block. 2 means after 1 ele, it jusmps to 3rd ele
      # check other args also
))

## Flattening

In [ ]:
# for flattening we again to need to call object of a class within add method of cnn object
# keras will automatically understand the input to flatten is output of all the convolutions and pooling which is strord in cnn object

cnn.add(tf.keras.layers.Flatten()) # no need to pass any args

## Step 4 : Full Connection

In [ ]:
# creatin fully connected layer to the above flatten layer
# remember from here, it is same as implementation of ANN

cnn.add(tf.keras.layers.Dense(units = 128,  # larger than one we taken for ANN as we are doing complex computation
                              activation = 'relu'
                              ))

## Step 5 : Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units = 1,  # as we are doing bianry classification and hence one class is needed
                              activation = 'sigmoid' # softmax for multiclass 
                              ))

# Part 3 : Training the CNN

## Compiling the CNN

In [ ]:
# compiling CNN here is same as that of ANN as we are deling with binary

cnn.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

# Training the CNN on Training set and evaluating it on test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 46s 145ms/step - loss: 0.6943 - accuracy: 0.5199 - val_loss: 0.6742 - val_accuracy: 0.5970
Epoch 2/25
250/250 [==============================] - 36s 143ms/step - loss: 0.6555 - accuracy: 0.6186 - val_loss: 0.6530 - val_accuracy: 0.6325
Epoch 3/25
250/250 [==============================] - 36s 143ms/step - loss: 0.6065 - accuracy: 0.6696 - val_loss: 0.6692 - val_accuracy: 0.6390
Epoch 4/25
250/250 [==============================] - 36s 145ms/step - loss: 0.5713 - accuracy: 0.7036 - val_loss: 0.5992 - val_accuracy: 0.6960
Epoch 5/25
250/250 [==============================] - 36s 144ms/step - loss: 0.5424 - accuracy: 0.7226 - val_loss: 0.5512 - val_accuracy: 0.7385
Epoch 6/25
250/250 [==============================] - 37s 146ms/step - loss: 0.5218 - accuracy: 0.7336 - val_loss: 0.5706 - val_accuracy: 0.7270
Epoch 7/25
250/250 [==============================] - 36s 146ms/step - loss: 0.4994 - accuracy: 0.7516 - val_loss: 0.6088 - val_ac

# Part 4 : Making a single Prediction

In [ ]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg',
                           target_size= (64, 64)  # we need to change the size of img as that of input
                           )
# We need to do some extraworks

# we need to change pil format (format of image) to a numpy array
test_image = image.img_to_array(test_image)

# predict method has to be called on precise format
# bacth size was mentioned so CNN is not trained to work on a single image
# so test image need to be in a batch of 32

test_image = np.expand_dims(test_image, axis = 0) # this makes the dimension to be added in first row as we need batch size 32 to be mentioned in first row
result = cnn.predict(test_image)

# but the output we get is either 0 or 1.
# we need to know whther dog is 0 or 1 and cat is 0 or 1
training_set.class_indices
# from here we get 1 corresponds to Dog

if result[0][0] == 1: # first 0 value indicates first index of batch 32. this is becasue we only have one image in batch of 32
                      # second 0 indicates the predicted output
  prediction = 'Dog'
else:
  prediction = 'Cat'



In [ ]:
print(prediction)

Dog


In [ ]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg',
                           target_size= (64, 64)  # we need to change the size of img as that of input
                           )

test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image, axis = 0) # this makes the dimension to be added in first row as we need batch size 32 to be mentioned in first row
result = cnn.predict(test_image)


training_set.class_indices


if result[0][0] == 1: # first 0 value indicates first index of batch 32. this is becasue we only have one image in batch of 32
                      # second 0 indicates the predicted output
  prediction = 'Dog'
else:
  prediction = 'Cat'



In [ ]:
print(prediction)

Cat
